# Cookbook
### LangChain Expression Language (LCEL)

---

Alejandro Ricciardi (Omegapy)  
created date: 01/21/2024   
[GitHub](https://github.com/Omegapy)  

Credit: [LangChain](https://python.langchain.com/docs/expression_language/)

<br>

--- 

Projects Description:  
**LangChain** is a framework for developing applications powered by language models. 

**In this project:**  I explore example of code for accomplishing common tasks with the LangChain Expression Language (LCEL). These examples show how to compose different Runnable (the core LCEL interface) components to achieve various tasks. If you're just getting acquainted with LCEL, the [Prompt + LLM](https://python.langchain.com/docs/expression_language/cookbook/prompt_llm_parser) section is a good place to start.

<p></p>
<b style="font-size:15;">
⚠️ This project requires an OpenAI key.
</b>


##### Project Map:  
- [API Keys](#api-keys)  
- [Prompt + LLM)](#prompt--llm)
    - [PromptTemplate + LLM](#prompttemplate--llm) 
        - [Base Example](#base-example-prompttemplate--llm)
        - [Attaching Stop Sequences](#attaching-stop-sequences)
        - [Attaching Function Call information](#attaching-function-call-information)
    - [PromptTemplate + LLM + OutputParser](#prompttemplate--llm--outputparser)
        - [Base Example](#base-example-prompttemplate--llm--outputparser)
        - [Functions Output Parser](#functions-output-parser)
    - [Simplifying input](#simplifying-input)
- [RAG](#rag)
    - [Base Example](#base-example-rag)
    - [Conversational Retrieval Chain (chat_message_history)](#conversational-retrieval-chain-chat_message_history)
        - [Base Example](#base-example-conversational-retrieval-chain)
        - [With Memory and returning source documents](#with-memory-and-returning-source-documents)
- [Multiple chains](#multiple-chains)
    - [Base Example](#base-example-multiple-chains)
        - [Using itemgetter](#using-itemgetter)
        - [Using RunnablePassthrough()](#using-runnablepassthrough)
    - [Branching and Merging](#branching-and-merging)
- [Querying a SQL DB](#querying-a-sql-db)
    - [Installing Chinook SQL DB](#installing-chinook-sql-db)
    - [Querying a SQL DB  Base Example](#querying-a-sql-db-base-example)
- [Agents](#agents)
- [Code writing (Agent Functions in Python)](#code-writing-agent-functions-in-python)
- [Routing by semantic similarity](#routing-by-semantic-similarity)
- [Adding memory to Chains](#adding-memory-to-chains)
- [Adding moderation](#adding-moderation)
- [Managing prompt size (Tokens)](#managing-prompt-size-tokens)

<br>

---


#### API Keys

In [2]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")

[Project Map](#project-map)

---

---
## Prompt + LLM

The most common and valuable composition is taking:

```PromptTemplate``` / ```ChatPromptTemplate``` -> ```LLM``` / ```ChatModel``` -> ```OutputParser```

Almost any other chains you build will use this building block.

<br>

---

### PromptTemplate + LLM

The simplest composition is just combining a prompt and model to create a chain that takes user input, adds it to a prompt, passes it to a model, and returns the raw model output.

Note, you can mix and match PromptTemplate/ChatPromptTemplates and LLMs/ChatModels as you like here.

#### Base Example (PromptTemplate + LLM)

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()

chain = prompt | model

chain.invoke({"foo": "bears"})

AIMessage(content="Sure, here's a bear-related joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they already have bear feet!")

#### Attaching Stop Sequences

In [3]:
chain = prompt | model.bind(stop=["\n"])
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?")

#### Attaching Function Call information

In [8]:
# The function is a structure in the JSON (JavaScript Object Notation) format. 
# JSON is widely used for storing and exchanging data and is language-agnostic, 
# meaning it can be used in various programming languages. 
# Note that the structure and syntax closely resemble how objects and arrays are defined in JavaScript or data structures in Python.
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]

chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}', 'name': 'joke'}})

[Project Map](#project-map)

---

### PromptTemplate + LLM + OutputParser
We can also add in an output parser to easily transform the raw LLM/ChatModel output into a more workable format

##### Base Example (PromptTemplate + LLM + OutputParser)

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

chain.invoke({"foo": "bears"})

'Sure, here\'s a bear joke for you:\n\nWhy don\'t bears have any money?\n\nBecause they always have "koala-fications" for the job!'

#### Functions Output Parser

In [14]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser # the 'functions' is writen in json

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions) # see functions at the beginning of the notebook
    | JsonOutputFunctionsParser() 
)

chain.invoke({"foo": "bears"})

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [11]:
# Output by key values
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

# Setup key
chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?"

In [12]:
# Punchline Key

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="punchline")
)

chain.invoke({"foo": "bears"})

'Because they already have bear feet!'

[Project Map](#project-map)

---

### Simplifying input
When you specify the function to return, you may just want to parse that directly

Two different code syntax how use ```RunnablePassthrough()```

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(foo=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke("bears")

"Why don't bears like fast food?"

In [16]:
chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke("bears")

"Why don't bears wear shoes?"

[Project Map](#project-map)

---

---
## RAG

“retrieval-augmented generation” chain

Also see RAG search in [Introduction to LangChain LCEL.ipynb](https://github.com/Omegapy/LLM-Frameworks-Tutorials/blob/1c99f3935c1b10478a4d7d2ba9b12556e88817a2/LangChain%20Tutorials/Tutorials%20from%20Langchain/Intro%20LangChain%20LCEL.ipynb)

<br>

---


### Base Example (RAG)

In [17]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [23]:
# vectorstore in RAM
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], 
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """
Answer the question based only on the following context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [24]:
# RunnablePassthrough()
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [25]:
# Using itemgetter
template = """
Answer the question based only on the following context: {context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

[Project Map](#project-map)

---

### Conversational Retrieval Chain (chat_message_history)

We can easily add in conversation history. This primarily means adding in chat_message_history

#### Base Example (Conversational Retrieval Chain)

In [26]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

In [27]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History: {chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [28]:
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [30]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [31]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)

_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}

conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [32]:
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

AIMessage(content='Harrison was employed at Kensho.')

In [33]:
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

AIMessage(content='Harrison worked at Kensho.')

[Project Map](#project-map)

---

#### With Memory and returning source documents

This shows how to use memory with the above. For memory, we need to manage that outside at the memory. For returning the retrieved documents, we just need to pass them through all the way.

In [34]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory

In [35]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [36]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [37]:
inputs = {"question": "where did harrison work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison was employed at Kensho.'),
 'docs': [Document(page_content='harrison worked at kensho')]}

In [38]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})

memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison was employed at Kensho.')]}

In [39]:
inputs = {"question": "but where did he really work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison actually worked at Kensho.'),
 'docs': [Document(page_content='harrison worked at kensho')]}

[Project Map](#project-map)

---

---
## Multiple chains
Runnables can easily be used to string together multiple Chains

<br>

---

### Base Example (Multiple chains)

##### Using itemgetter

In [3]:
from operator import itemgetter

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

'El país en el que se encuentra la ciudad de Honolulu, Hawái, donde nació Barack Obama, el 44º presidente de Estados Unidos, es Estados Unidos.'

##### Using RunnablePassthrough()

In [4]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [5]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Coral. and the flag of Comoros?')])

In [6]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color commonly associated with "Coral" is a shade of pink or orange. The flag of Comoros consists of four horizontal stripes of yellow, white, red, and blue, from top to bottom.')

[Project Map](#project-map)

---

### Branching and Merging
You may want the output of one component to be processed by 2 or more other components. [RunnableParallels](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html#langchain_core.runnables.base.RunnableParallel) let you split or fork the chain so multiple components can process the input in parallel. Later, other components can join or merge the results to synthesize a final response. This type of chain creates a computation graph that looks like the following:

In [8]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

chain.invoke({"input": "scrum"})

'While Scrum has its drawbacks, it is important to consider these cons in the context of the project and organization. Many of the perceived negatives can be mitigated with proper implementation and by addressing potential challenges.\n\nFor example, time and resource constraints can be managed by setting realistic expectations and ensuring that teams have the necessary resources and support. While detailed documentation may not be a primary focus in Scrum, it is still important to maintain sufficient records and documentation to track project progress and decisions.\n\nWhile Scrum may have limitations for large-scale projects, it can still be adapted and scaled by implementing frameworks such as Scrum of Scrums or using agile scaling frameworks like SAFe (Scaled Agile Framework) or LeSS (Large-Scale Scrum). These frameworks provide additional structure and coordination for managing large and complex initiatives.\n\nTo address the dependency on a skilled Scrum Master, organizations can

[Project Map](#project-map)

---

---
## Querying a SQL DB

<br>

---

### Installing Chinook SQL DB

We’ll need the Chinook sample DB for this example. There’s many places to download it from, e.g. https://database.guide/2-sample-databases-sqlite/

**The Chinook Database**
The Chinook database was created as an alternative to the Northwind database. 
It represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

The Chinook database is available on [GitHub](https://github.com/lerocha/chinook-database/tree/master/ChinookDatabase). 
It’s available for various DBMSs including MySQL, SQL Server, SQL Server Compact, PostgreSQL, Oracle, DB2, and of course, SQLite.

Install the Chinook Database
You can install the Chinook database in SQLite by running the SQL script available on GitHub. 
It’s quite a large script, so you might find it easier to run it from a file.

First, save the [Chinook_Sqlite.sql](https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql) script to a folder/directory on your computer. That’s a direct link to the script on GitHub.

Now create a database called Chinook. You can do this by connecting to SQLite with the following
Note: Running this script creates the database tables and populates them with data.

In the terminal:
```
sqlite3 Chinook.db 
sqlite>
sqlite> .read Chinook_Sqlite.sql
```
verify that it created the database
```
sqlite> SELECT * FROM Artist LIMIT 10;
1|AC/DC
2|Accept
3|Aerosmith
4|Alanis Morissette
5|Alice In Chains
6|Antônio Carlos Jobim
7|Apocalyptica
8|Audioslave
9|BackBeat
10|Billy Cobham
sqlite>
```

### Querying a SQL DB Base Example

In [9]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Based on the table schema below, write a SQL query that would answer the user's question: {schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [10]:
from langchain_community.utilities import SQLDatabase

In [21]:
db = SQLDatabase.from_uri("sqlite:///./data/Chinook.db")

In [22]:
def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) FROM Employee;'

In [24]:
template = """
Based on the table schema below, question, sql query, and sql response, write a natural language response: {schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

In [25]:
full_chain.invoke({"question": "How many employees are there?"})

AIMessage(content='There are 8 employees in the database.')

[Project Map](#project-map)

---

---
## Agents
You can pass a Runnable into an agent.

<br>

---

In [26]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain_community.chat_models import ChatOpenAI

In [27]:
model = ChatOpenAI(model="gpt-4")

In [28]:
@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

In [29]:
tool_list = [search]

In [30]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo") # https://smith.langchain.com/hub/hwchase17/xml-agent-convo?organizationId=bd3fb686-7fb6-557c-95f9-8f1d8dcb7aee

In [31]:
# Logic for going from intermediate steps to a string to pass into model
# This is pretty tied to the prompt
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log


# Logic for converting tools to string to go in prompt
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

Building an agent from a runnable usually involves a few things:

1. Data processing for the intermediate steps. These need to represented in a way that the language model can recognize them. This should be pretty tightly coupled to the instructions in the prompt
2. The prompt itself
3. The model, complete with stop tokens if needed
4. The output parser - should be in sync with how the prompt specifies things to be formatted.

In [32]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

In [33]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [35]:
agent_executor.invoke({"input": "whats the weather in New york?"})



> Entering new AgentExecutor chain...
<tool>search</tool><tool_input>weather in New York32 degrees<final_answer>The weather in New York is 32 degrees

> Finished chain.


{'input': 'whats the weather in New york?',
 'output': 'The weather in New York is 32 degrees'}

[Project Map](#project-map)

---

---
## Code writing (Agent Functions in Python)
Example of how to use LCEL to write Python code.

<br>

---

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL # pip install langchain_experimental
from langchain_openai import ChatOpenAI

In [37]:
template = """
Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```
"""

prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI()

In [38]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

In [39]:
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [40]:
chain.invoke({"input": "whats 2 plus 2"})

Python REPL can execute arbitrary code. Use with caution.


'4\n'

[Project Map](#project-map)

---

---
## Routing by semantic similarity
With LCEL you can easily add [custom routing logic](https://python.langchain.com/docs/expression_language/how_to/routing#using-a-custom-function) to your chain to dynamically determine the chain logic based on user input. All you need to do is define a function that given an input returns a ```Runnable```.

One especially useful technique is to use embeddings to route a query to the most relevant prompt. Here’s a very simple example.

<br>

---

In [41]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [43]:
chain.invoke("What's a black hole")

Using PHYSICS


'A black hole is a region in space where gravity is extremely strong, so strong that nothing, not even light, can escape its gravitational pull. It is formed when a massive star collapses under its own gravity, creating a point of infinite density called a singularity. The event horizon, which is the boundary of a black hole, is the point of no return beyond which anything that enters will be trapped forever. Black holes have fascinated scientists for decades, and studying them helps us understand the nature of gravity and the behavior of matter in extreme conditions.'

In [42]:
print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where gravity is extremely strong, to the point that nothing, not even light, can escape its gravitational pull. It is formed when a massive star collapses under its own gravitational force upon running out of fuel. The collapse is so intense that it forms a singularity, a point of infinite density at the center of the black hole, surrounded by an event horizon, which is the boundary beyond which nothing can escape. Black holes have fascinated scientists and are still being actively studied to better understand their properties and effects on the surrounding space.


In [44]:
print(chain.invoke("What's a path integral"))

Using MATH
A path integral is a concept in mathematics and physics, specifically in the field of quantum mechanics. It is used to describe the behavior of quantum particles moving between two points in space and time.

In simple terms, a path integral considers all possible paths that a particle can take from an initial point to a final point. Each path is assigned a weight or probability amplitude, which is determined by the action of the particle along that path. The action is a mathematical quantity that depends on the path's shape and the physical laws governing the system.

To calculate a path integral, one sums up the contributions from all possible paths, taking into account their respective probability amplitudes. This summation involves integrating over all possible configurations of the particle's position and momentum at each point along the paths.

The path integral concept is a powerful tool that allows us to analyze the quantum behavior of particles in a wide range of phy

[Project Map](#project-map)

---

---
## Adding memory to Chains
This shows how to add memory to an arbitrary chain. Right now, you can use the memory classes but need to hook it up manually

<br>

---

In [45]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [46]:
memory = ConversationBufferMemory(return_messages=True)

In [47]:
memory.load_memory_variables({})

{'history': []}

In [48]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [49]:
inputs = {"input": "hi im bob"}
response = chain.invoke(inputs)
response

AIMessage(content='Hello Bob! How can I assist you today?')

In [51]:
memory.save_context(inputs, {"output": response.content})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi im bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')]}

In [52]:
inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
response

AIMessage(content='Your name is Bob.')

[Project Map](#project-map)

---

---
## Adding moderation
This shows how to add in moderation (or other safeguards) around your LLM application.

<br>

---

In [53]:
from langchain.chains import OpenAIModerationChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAI

In [54]:
moderate = OpenAIModerationChain()

In [55]:
model = OpenAI()
prompt = ChatPromptTemplate.from_messages([("system", "repeat after me: {input}")])

In [57]:
chain = prompt | model

In [58]:
chain.invoke({"input": "you are stupid"})

'\n\nI am an AI and do not have the capacity to feel emotions or have thoughts about my intelligence. I am simply programmed to assist and provide information to the best of my abilities.'

In [59]:
moderated_chain = chain | moderate

In [60]:
moderated_chain.invoke({"input": "you are stupid"})

APIRemovedInV1: 

You tried to access openai.Moderation, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


[Project Map](#project-map)

---

---
## Managing prompt size (Tokens)
Agents dynamically call tools. The results of those tool calls are added back to the prompt, so that the agent can plan the next action. Depending on what tools are being used and how they’re being called, the agent prompt can easily grow larger than the model context window.

With LCEL, it’s easy to add custom functionality for managing the size of prompts within your chain or agent. Let’s look at simple agent example that can search Wikipedia for information.

<br>

---

In [63]:
%pip install --upgrade --quiet  langchain langchain-openai wikipedia

Note: you may need to restart the kernel to use updated packages.


In [61]:
from operator import itemgetter

from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts.chat import ChatPromptValue
from langchain.tools import WikipediaQueryRun
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

In [64]:
wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=10_000)
)
tools = [wiki]

In [65]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo")

Let’s try a many-step question without any prompt size handling:

In [66]:
agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `List of presidents of the United States`


Page: List of presidents of the United States
Summary: The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. Since the office was established in 1789, 45 men have served in 46 presidencies. The first president, George Washington, won a unanimous vote of the Electoral College. Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the number of presidencies and the number of individuals who have served as president. The incumbent president is Joe Biden.The presidency of William Henry Harrison, who died 31 days after taking 

Traceback (most recent call last):
  File "C:\Users\User\.virtualenvs\Tutorials_from_Langchain-HvUJzvlK\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_7180\2448814465.py", line 14, in <module>
    agent_executor.invoke(
  File "C:\Users\User\.virtualenvs\Tutorials_from_Langchain-HvUJzvlK\Lib\site-packages\langchain\chains\base.py", line 162, in invoke
    raise e
  File "C:\Users\User\.virtualenvs\Tutorials_from_Langchain-HvUJzvlK\Lib\site-packages\langchain\chains\base.py", line 156, in invoke
    self._call(inputs, run_manager=run_manager)
  File "C:\Users\User\.virtualenvs\Tutorials_from_Langchain-HvUJzvlK\Lib\site-packages\langchain\agents\agent.py", line 1376, in _call
    next_step_output = self._take_next_step(
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\.virtualenvs\Tutorials_from_Langchain-HvUJzvlK\Lib\site-packages\langch

[LangSmith trace](https://smith.langchain.com/public/60909eae-f4f1-43eb-9f96-354f5176f66f/r)

Unfortunately we run out of space in our model’s context window before we the agent can get to the final answer. Now let’s add some prompt handling logic. To keep things simple, if our messages have too many tokens we’ll start dropping the earliest AI, Function message pairs (this is the model tool invocation message and the subsequent tool output message) in the chat history.

In [67]:
def condense_prompt(prompt: ChatPromptValue) -> ChatPromptValue:
    messages = prompt.to_messages()
    num_tokens = llm.get_num_tokens_from_messages(messages)
    ai_function_messages = messages[2:]
    while num_tokens > 4_000:
        ai_function_messages = ai_function_messages[2:]
        num_tokens = llm.get_num_tokens_from_messages(
            messages[:2] + ai_function_messages
        )
    messages = messages[:2] + ai_function_messages
    return ChatPromptValue(messages=messages)


agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | condense_prompt
    | llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `List of presidents of the United States`


Page: List of presidents of the United States
Summary: The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. Since the office was established in 1789, 45 men have served in 46 presidencies. The first president, George Washington, won a unanimous vote of the Electoral College. Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the number of presidencies and the number of individuals who have served as president. The incumbent president is Joe Biden.The presidency of William Henry Harrison, who died 31 days after taking 

C:\Users\User\.virtualenvs\Tutorials_from_Langchain-HvUJzvlK\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\User\.virtualenvs\Tutorials_from_Langchain-HvUJzvlK\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: American goldfinch
Summary: The American goldfinch (Spinus tristis) is a small North American bird in the finch family. It is migratory, ranging from mid-Alberta to North Carolina during the breeding season, and from just south of the Canada–United States border to Mexico during the winter.
The only finch in its subfamily to undergo a complete molt, the American goldfinch displays sexual dichromatism: the male is a vibrant yellow in the summer and an olive color during the winter, while the female is a dull yellow-brown shade which brightens only slightly during the summer. The male displays brightly colored plumage during the breeding season to attract a mate.
The American goldfinch is a granivore and adapted for the consumption of seedheads, with a conical beak to remove the seeds and agile feet to grip the stems of seedheads while feeding. It is a social bird and will gather in large flocks while feeding and migrating. It may behave territorially during nest construction, but 

{'input': "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?",
 'output': 'The current US president is Joe Biden. His home state is Delaware. The state bird of Delaware is the American goldfinch (Spinus tristis). The scientific name for the American goldfinch is Spinus tristis.'}